In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import json
# !pip install wordcloud
from wordcloud import WordCloud
import pickle
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
pip install google-play-scraper

In [ ]:
trashcare = pd.read_csv('/content/drive/MyDrive/KULIAH/Semester5/Pemrograman sistem Cerdas/Prak_AdiSucipto/Sentimen Analisis/trashcare.csv') #variable data berisikan data
trashcare

In [ ]:
trashcare.head()

In [ ]:
trashcare.shape

In [ ]:
trashcare = trashcare.drop('Unnamed: 0', axis=1)

In [ ]:
trashcare.columns

In [ ]:
trashcare.head()

In [ ]:
trashcare.to_csv('/content/drive/MyDrive/KULIAH/Semester5/Pemrograman sistem Cerdas/Prak_AdiSucipto/Sentimen Analisis/trashcare.csv')

In [ ]:
trashcare = pd.read_csv('/content/drive/MyDrive/KULIAH/Semester5/Pemrograman sistem Cerdas/Prak_AdiSucipto/Sentimen Analisis/trashcare.csv', index_col = 0)

In [ ]:
trashcare.tail()

Preprocessing

In [ ]:
trashcare["score"].value_counts()

In [ ]:
text = trashcare["content"].tolist()
print(len(text))

In [ ]:
y = trashcare["score"]
y = to_categorical(y)
print(y)

In [ ]:
def emoji(review):
    # Smile -- :), : ), :-), (:, ( :, (-:, :') , :O
    review = re.sub(r'(:\s?\)|:-\)|\(\s?:|\(-:|:\'\)|:O)', ' positiveemoji ', review)
    # Laugh -- :D, : D, :-D, xD, x-D, XD, X-D
    review = re.sub(r'(:\s?D|:-D|x-?D|X-?D)', ' positiveemoji ', review)
    # Love -- <3, :*
    review = re.sub(r'(<3|:\*)', ' positiveemoji ', review)
    # Wink -- ;-), ;), ;-D, ;D, (;,  (-; , @-)
    review = re.sub(r'(;-?\)|;-?D|\(-?;|@-\))', ' positiveemoji ', review)
    # Sad -- :-(, : (, :(, ):, )-:, :-/ , :-|
    review = re.sub(r'(:\s?\(|:-\(|\)\s?:|\)-:|:-/|:-\|)', ' negetiveemoji ', review)
    # Cry -- :,(, :'(, :"(
    review = re.sub(r'(:,\(|:\'\(|:"\()', ' negetiveemoji ', review)
    return review

In [ ]:
import re

def process_review(review):
    review = review.lower()                                             # Huruf kecil string
    review = re.sub(r"\d+", " ", str(review))                           # Menghapus semua digit
    review = re.sub('&quot;'," ", review)                               # Hapus (&quot;)
    review = emoji(review)                                              # Menggantikan Emoji
    review = re.sub(r"\b[a-zA-Z]\b", "", str(review))                   # Menghapus semua karakter tunggal
    for word in review.split():
        if word.lower() in contractions:
            review = review.replace(word, contractions[word.lower()])   # Menggantikan kontraksi
    review = re.sub(r'(.)\1+', r'\1\1', review)                         # Ubah lebih dari 2 pengulangan huruf menjadi 2 huruf
    review = re.sub(r"\s+", " ", str(review))                           # Replaces double spaces with single space
    return review

In [ ]:
trashcare.head()

Tokenization

In [ ]:
token = Tokenizer()
token.fit_on_texts(text)

In [ ]:
token.index_word

In [ ]:
vocal = len(token.index_word)+1
vocal

Case Folding

In [ ]:
trashcare = trashcare.copy()

In [ ]:
trashcare.head()

In [ ]:
for i in range(len(trashcare)):
    if type(trashcare["content"][i]) != str:
        print(str(i), "Bukan string")

In [ ]:
# Case Folding

def clean_content(content):
    import string, re

    content = content.lower() # menjadikan lowercase
    content = re.sub("[^a-z]", " ", content) # hapus semua karakter kecuali a-z
    content = re.sub("\t", " ", content) # mengganti tab dengan spasi
    content = re.sub("\n", " ", content) # mengganti new line dengan spasi
    content = re.sub("\s+", " ", content) # mengganti spasi > 1 dengan 1 spasi
    content = content.strip() # menghapus spasi di awal dan akhir

    return content

In [ ]:
trashcare["content_clean"] = trashcare["content"].apply(clean_content)
trashcare.head()

Stopword Removal

In [ ]:
# Sastrawi
pip install Sastrawi
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StopWordRemoverFactory()
stopword_sastrawi = factory.get_stop_words()
len(stopword_sastrawi)

In [ ]:
# NLTK
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

In [ ]:
stopword_nltk = set(stopwords.words("indonesian"))
len(stopword_nltk)

In [ ]:
df1 = pd.DataFrame(stopword_nltk)
df2 = pd.DataFrame(stopword_sastrawi)

common = df1.merge(df2, on = [0], how = "left")
print(common)

ga_ada = df2[(~df2[0].isin(common[0]))] # ~ -> negasi (True jadi False)
ga_ada

In [ ]:
def clean_stopword(content):
    # Stopword Sastrawi
    from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

    factory = StopWordRemoverFactory()
    stopword_sastrawi = factory.get_stop_words()

    content = content.split() # split jadi kata per kata
    content = [w for w in content if w not in stopword_sastrawi] # hapus stopwords
    content = " ".join(w for w in content) # join semua kata yang bukan stopwords

    # Stopword NLTK
    import nltk
    #nltk.download()
    from nltk.corpus import stopwords

    stopword_nltk = set(stopwords.words("indonesian"))
    stopword_nltk = stopword_nltk

    content = content.split() # split jadi kata per kata
    content = [w for w in content if w not in stopword_nltk] # hapus stopwords
    content = " ".join(w for w in content) # join semua kata yang bukan stopwords

    return content

In [ ]:
trashcare["content_clean"] = trashcare["content_clean"].apply(clean_stopword)
trashcare.head()

Stemming